## Online Retail Data set 기반으로 고객 세그멘테이션을 위한 군집분석을 다음 사항을 준수하여 수행하세요.
- 목표 :
  - 타깃 마케팅 : 고객을 여러 특성에 맞게 세분화해서 그 유형에 따라 맞춤형 마케팅이나 서비스를 제공.
  - 고객의 상품 구매 이력 데이터 기반에 기초한 타깃 마케팅
- 군집화 방법 :기본적인 고객 분석 요소인 RFM 기법을 이용해서 고객을 군집화
  - RECENCY(R): 가장 최근 상품 구입 일에서 오늘까지의 기간(최근에 구매했느냐에 따라 고객의 로열티가 다르겠죠) Recency 칼럼은 오늘 날짜에서 주문 일자를 뺀 날짜, 오늘 날짜를 2011년 12월 10일로 설정
  - FREQUENCY(F): 상품 구매 횟수(고객 밸류에서 다르겠죠)
  - MONETARY VALUE(M) : 총 구매 금액(작은 금액을 구매하는 고객도 있겠고 엄청나게 큰 금액을 구매하는 고객도 있음)
- 데이터셋
  - UCI 에서 제공하는 Online Retail Data set
  - 데이터셋 칼럼 구성
    - InvoiceNo: 주문번호, ‘C’로 시작하는 것은 취소 주문
    - StockCode: 제품 코드(Item Code)
    - Description: 제품 설명
    - Quantity : 주문 제품 건수
    - InvoiceDate: 주문 일자
    - UnitPrice: 제품 단가 ( 제품 단가 X 제품 건수 = 가격 )
    - CustomerID: 고객 번호
    - Country: 국가명(주문 고객의 국적)

- 분석 가이드
  - 데이터 세트 로딩과 데이터 클렌징
    * Null 데이터 제거:  CustomerID의 Null 값이 많은 것으로 확인됨. 고객 세그멘이션을 수행하므로 고객 식별 번호가 없는 데이터는 삭제
    * 오류 데이터 삭제: 대표적인 오류 데이터는 Quantity 또는 UnitPrice 가 0보다 작은 경우. Quantity 가 0보다 작은 경우는 반환을 뜻하는 값으로 InvoiceNO 앞자리가 ‘C’로 되어있음. 분석의 효율성을 위해 삭제    
    * Country 칼럼의 주요 주문 고객 국가인 영국 데이터만을 필터링하여 선택
  - RFM 기반 데이터 가공
    * UnitPrice’(제품 단가), ‘Quantity’(주문 제품 건수)를 곱해서 주문 금액 칼럼 생성, CusmtomerID int형으로 형변환
    * 주문 금액, 주문 횟수가 특정 고객에게 과도하게 되어있는지 탐색.<br>
    * CustomerID로 group by 하여 TOP-5 구매 횟수 고객 ID 추출<br>
    * CustomerID로 group by 하고 주문 금액 칼럼을 더하여 TOP-5 주문 금액 고객 ID 추출
    * 온라인 데이터 세트에 대해서 ‘InvocieNO(주문번호)’ + ‘StockCode(상품코드)’ 거의 1에 가까운 유일한 식별자 레벨로 되어있는지 확인
    * 주문번호 기준의 데이터를 개별 고객 기준의 데이터로 변환 - 고객 레벨로 주문 기간, 주문 횟수, 주문 금액 데이터를 기반으로 하여 세그멘테이션을 수행하기 위한 데이터 처리
    * 개별 고객 기준의 데이터 셋은 CustomerID, Recency, Frequency, Monetary 4개의 칼럼으로 구성
  - RFM 기반 고객 세그멘테이션
    * 온라인 판매 데이터는 소매업체로 추정되는 특정 고객의 대규모 주문이 포함되어 있어 주문 횟수와 주문 금액에서 개인 고객 주문과 매우 큰 차이가 있음을 왜곡된 데이터 분포도를 시각화하여 확인
    * 왜곡 정도가 매우 높은 데이터 세트에 K-평균을 적용하면 중심의 개수를 증가시키더라도 변별력이 떨어지는 군집화가 수행되기 때문에 StandScaler로 평균과 표준편차를 재조정한 후 시각화하여 군집화 결과를 확인. 시각화는 제공된 사용자 함수 이용하며 군집 개수를 2 ~ 5개 변화시키면서 수행
      - visualize_silhouette(cluster_lists, X_features)
      - visualize_kmeans_plot_multi(cluster_lists, X_features)    
    
    * 데이터 세트의 왜곡정도를 낮추기 위해 로그변환 이용하여 다시 시각화하여 군집화 결과를 확인
    * 3개 군집의 특징을 통계적 방법을 통해서 해석 및 활용 방안 수립

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/hjh_kita_directory/Github/kita_231026/m5_ml/Online_Retail.xlsx')
print(df.shape)
df.head()

Mounted at /content/drive
(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/hjh_kita_directory/Github/kita_231026/m5_ml/Online_Retail.xlsx')

In [2]:
df.info()
# 고객에 대한 구분이 없는 것은 의미가 없음
# 고객에 대한 ID가 없는것들은, customer segmentation에는 필요가 없음.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [ ]:
# 널값 확인.
null_counts = df.isnull().sum()
print(null_counts)
# 고객 ID 널값이 있는데.. 삭제해야함 -> 고객의 타깃 마케팅을 하는건데 ID 가없으면 의미가 없으니 널값되어있는 행은 삭제.

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


데이터 세트의 전체 건수, 칼럼 타입, Null 개수 확인
- 전체 데이터 541,909개
- CustomerID의 Null 값은 13만 5천건, 너무 많은 것으로 확인

In [4]:
# 데이터클렌징.
# 그리고 Quantity(주문 제품 건수), UnitPrice(제품 단가) 는 0보다 작은값이 나오면 안됨
# Country - value.counts 로 보면 대부분 영국 데이터만을 선택해가지고 할것임
# invoice no. .. -> CustomerID, Recency, Frequency, Monetary 컬럼 4개로 구성해서 고객 세그멘테이션을 기대할수있음

- 대표적인 오류 데이터는 Quantity 또는 UnitPrice 가 0보다 작은 경우. Quantity 가 0보다 작은 경우는 반환을 뜻하는 값으로 InvoiceNo 앞자리 가 'C'로 되어있음. 분석의 효율성을 위해 삭제
- 고객 세그멘테이션을 수행하므로 고객 식별 번호가 없는 데이터는 필요가 없기에 삭제

In [ ]:
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]
df = df[df['CustomerID'].notnull()]
print(df.shape)
df.isnull().sum()

In [ ]:
# Null 데이터 제거
df = df.dropna(subset=['CustomerID'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [ ]:
# 오류 데이터 삭제
df = df[df['Quantity'] >= 0] # Quantity(주문 제품 건수) 는 0 보다 작을 수 없음
df = df[df['UnitPrice'] >= 0] # UnitPrice(제품 단가) 는 0 보다 작을 수 없음
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      397924 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.3+ MB


In [ ]:
# Country -> 제일 많은 영국만으로 할것임. 그외 삭제
df['Country'].value_counts()

United Kingdom          354345
Germany                   9042
France                    8342
EIRE                      7238
Spain                     2485
Netherlands               2363
Belgium                   2031
Switzerland               1842
Portugal                  1462
Australia                 1185
Norway                    1072
Italy                      758
Channel Islands            748
Finland                    685
Cyprus                     614
Sweden                     451
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     248
Unspecified                244
Singapore                  222
Iceland                    182
USA                        179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
European Community          60
RSA                         58
Lebanon                     45
Lithuani

In [ ]:
df = df[df['Country'] == 'United Kingdom']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354345 entries, 0 to 541893
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    354345 non-null  object        
 1   StockCode    354345 non-null  object        
 2   Description  354345 non-null  object        
 3   Quantity     354345 non-null  int64         
 4   InvoiceDate  354345 non-null  datetime64[ns]
 5   UnitPrice    354345 non-null  float64       
 6   CustomerID   354345 non-null  float64       
 7   Country      354345 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.3+ MB


In [ ]:
# InvoiceNO 앞자리가 ‘C’-> 삭제.(취소주문)
df = df[~df['InvoiceNo'].astype(str).str.contains('^C')] # 정규표현식. ^C -> C로 시작하는 경우
df.info()
# 근데..변화가 없음. 위의 과정에서 함께 삭제가 되었나봄..

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354345 entries, 0 to 541893
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    354345 non-null  object        
 1   StockCode    354345 non-null  object        
 2   Description  354345 non-null  object        
 3   Quantity     354345 non-null  int64         
 4   InvoiceDate  354345 non-null  datetime64[ns]
 5   UnitPrice    354345 non-null  float64       
 6   CustomerID   354345 non-null  float64       
 7   Country      354345 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.3+ MB


In [ ]:
# UnitPrice’(제품 단가), ‘Quantity’(주문 제품 건수)를 곱해서 주문 금액 칼럼 생성
# 컬럼 명은 'OrderAmount'
df['OrderAmount'] = df['UnitPrice']*df['Quantity']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354345 entries, 0 to 541893
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    354345 non-null  object        
 1   StockCode    354345 non-null  object        
 2   Description  354345 non-null  object        
 3   Quantity     354345 non-null  int64         
 4   InvoiceDate  354345 non-null  datetime64[ns]
 5   UnitPrice    354345 non-null  float64       
 6   CustomerID   354345 non-null  float64       
 7   Country      354345 non-null  object        
 8   OrderAmount  354345 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 27.0+ MB


In [ ]:
# CusmtomerID int형으로 형변환
df['CustomerID'] = df['CustomerID'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354345 entries, 0 to 541893
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    354345 non-null  object        
 1   StockCode    354345 non-null  object        
 2   Description  354345 non-null  object        
 3   Quantity     354345 non-null  int64         
 4   InvoiceDate  354345 non-null  datetime64[ns]
 5   UnitPrice    354345 non-null  float64       
 6   CustomerID   354345 non-null  int64         
 7   Country      354345 non-null  object        
 8   OrderAmount  354345 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 27.0+ MB


In [ ]:
# 주문 금액, 주문 횟수가 특정 고객에게 과도하게 되어있는지 탐색.
# CustomerID로 group by 하여 TOP-5 구매 횟수 고객 ID 추출
# 주문 횟수 -> order_counts
order_counts = df.groupby('CustomerID')['InvoiceNo'].nunique()
order_counts

CustomerID
12346      1
12747     11
12748    210
12749      5
12820      4
        ... 
18280      1
18281      1
18282      2
18283     16
18287      3
Name: InvoiceNo, Length: 3921, dtype: int64

In [ ]:
# 구매 횟수 Top-5
order_counts = order_counts.sort_values(ascending=False)
order_counts_top5 = order_counts.head(5)
order_counts_top5

CustomerID
12748    210
17841    124
13089     97
14606     93
15311     91
Name: InvoiceNo, dtype: int64

In [ ]:
# CustomerID로 group by 하고 주문 금액 칼럼을 더하여 TOP-5 주문 금액 고객 ID 추출
# 주문 금액 -> 'OrderAmount'
total_amount = df.groupby('CustomerID')['OrderAmount'].sum()
total_amount = total_amount.sort_values(ascending=False)
total_amount_top5 = total_amount.head(5)
total_amount_top5

CustomerID
18102    259657.30
17450    194550.79
16446    168472.50
17511     91062.38
16029     81024.84
Name: OrderAmount, dtype: float64

In [ ]:
# 온라인 데이터 세트에 대해서 'InvocieNO(주문번호)' + 'StockCode(상품코드)' 거의 1에 가까운 유일한 식별자 레벨로 되어있는지 확인
# 'InvoiceNo'와 'StockCode'의 조합으로 새로운 식별자 열 생성
df['Identifier'] = df['InvoiceNo'].astype(str) + '-' + df['StockCode'].astype(str)

# 새로운 식별자 열의 고유값 개수 계산
unique_identifier_count = df['Identifier'].nunique()

# 전체 데이터 행의 개수
total_rows = len(df)

# 고유값 개수와 전체 행의 개수를 출력하여 비교
print("고유 식별자 개수:", unique_identifier_count)
print("전체 데이터 행의 개수:", total_rows)

# 유일한 식별자 역할을 하는지 확인
if unique_identifier_count / total_rows > 0.99:  # 99% 이상이면 거의 유일한 것으로 간주
    print("‘InvoiceNo’와 ‘StockCode’의 조합은 데이터셋에서 거의 유일한 식별자 역할을 합니다.")
else:
    print("‘InvoiceNo’와 ‘StockCode’의 조합은 유일한 식별자 역할을 하지 않습니다.")


고유 식별자 개수: 344455
전체 데이터 행의 개수: 354345
‘InvoiceNo’와 ‘StockCode’의 조합은 유일한 식별자 역할을 하지 않습니다.


군집 개수별로 군집화 구성을 시각화하는 사용자 함수

In [ ]:
### 여러개의 클러스터링 갯수를 List로 입력 받아 각각의 클러스터링 결과를 시각화
def visualize_kmeans_plot_multi(cluster_lists, X_features):

    from sklearn.cluster import KMeans
    from sklearn.decomposition import PCA
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import warnings
    warnings.filterwarnings('ignore')

    # plt.subplots()으로 리스트에 기재된 클러스터링 만큼의 sub figures를 가지는 axs 생성
    n_cols = len(cluster_lists)
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)

    # 입력 데이터의 FEATURE가 여러개일 경우 2차원 데이터 시각화가 어려우므로 PCA 변환하여 2차원 시각화
    pca = PCA(n_components=2)
    pca_transformed = pca.fit_transform(X_features)
    dataframe = pd.DataFrame(pca_transformed, columns=['PCA1','PCA2'])

     # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 KMeans 클러스터링 수행하고 시각화
    for ind, n_cluster in enumerate(cluster_lists):

        # KMeans 클러스터링으로 클러스터링 결과를 dataframe에 저장.
        clusterer = KMeans(n_clusters = n_cluster, n_init='auto', max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(pca_transformed)
        dataframe['cluster']=cluster_labels

        unique_labels = np.unique(clusterer.labels_)
        markers=['o', 's', '^', 'x', '*']

        # 클러스터링 결과값 별로 scatter plot 으로 시각화
        for label in unique_labels:
            label_df = dataframe[dataframe['cluster']==label]
            if label == -1:
                cluster_legend = 'Noise'
            else :
                cluster_legend = 'Cluster '+str(label)
            axs[ind].scatter(x=label_df['PCA1'], y=label_df['PCA2'], s=70, edgecolor='k', marker=markers[label], label=cluster_legend)
            # axs[ind].scatter(x=label_df['PCA1'], y=label_df['PCA2'], s=70, marker='o', edgecolor='k', facecolor='none')

        axs[ind].set_title('Number of Cluster : '+ str(n_cluster))
        axs[ind].legend(loc='upper right')

    plt.show()